In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import LdaModel
from gensim import models, corpora, similarities
from gensim.parsing.preprocessing import STOPWORDS

import re
from nltk.stem import WordNetLemmatizer
import time
from nltk import FreqDist
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [53]:
table = pd.read_csv('wikipedia_articles.csv')

In [37]:
table.tail()

,Title,URL,Text,Table Table
11416,Circus,https://en.wikipedia.org/wiki/Circus_(Britney_...,\n\n\nCircus\n is the sixth studio album by Am...,Circus\nStandard edition. The deluxe edition f...
11417,Character (symbol),https://en.wikipedia.org/wiki/Character_(symbol),A \ncharacter\n is a \nsign\n or \nsymbol\n.\n...,\n\n\n\n\n\nLook up \ncharacter\n in Wiktionar...
11418,Three Hearts,https://en.wikipedia.org/wiki/Three_Hearts_(film),Three Hearts\n (\nFrench\n: \n3 cœurs\n) is a ...,Three Hearts\nTheatrical release poster\nDirec...
11419,Gryllinae,https://en.wikipedia.org/wiki/Gryllinae,"\n\nMany, see text\n\nGryllinae\n, or \nfield ...",\n\n\n\n\nGryllinae\n\n\n\n\n\n\n\n\n\n\n\nCom...
11420,Gryllus campestris,https://en.wikipedia.org/wiki/Gryllus_campestris,"\n\nGryllus campestris\n, the \nEuropean field...",\n\n\n\n\nGryllus campestris\n\n\n\n\n\n\n\n\n...


In [38]:
table[table.duplicated(subset=['Title'])]

,Title,URL,Text,Table Table
8,Virus,https://en.wikipedia.org/wiki/Virion,\n\n\n\nSee text\n\n\nA \nvirus\n is a submicr...,\n\n\n\n\nVirus\n\n\n\n\n\n\n\n\n\n\n\nSARS-Co...
49,Recombinant DNA,https://en.wikipedia.org/wiki/Gene_splicing,Recombinant DNA\n (\nrDNA\n) molecules are \nD...,Library resources\n about \n \nRecombinant pro...
395,Hoshū jugyō kō,https://en.wikipedia.org/wiki/Hosh%C5%AB_jugy%...,"Hoshū jugyō kō\n (\n補習授業校\n)\n, or \nhoshūkō\n...",This section \nneeds expansion\n. \nYou can he...
493,Twisted pair,https://en.wikipedia.org/wiki/Twisted-pair_cable,\n\nTwisted pair\n cabling is a type of wiring...,\n\nCommon industry nomenclature for cable con...
509,Token ring,https://en.wikipedia.org/wiki/Token_Ring,Token Ring\n is a \ncomputer networking\n tech...,\n\n\n\n\nStart Delimiter\n\n\n\nAccess Contro...
916,HMS,https://en.wikipedia.org/wiki/HMS_St_Michael_(...,\n\n\nHMS \nSt Michael\n was a 90-gun \nsecond...,\n\n\n\n\nMarlborough\n after the \nBattle of ...
1013,"Henri de La Tour d'Auvergne, Viscount of Turenne",https://en.wikipedia.org/wiki/Turenne,War of Devolution\n\n\nHenri de La Tour d'Auve...,"Henri de La Tour d’Auvergne, Viscount of Turen..."
1088,ICE 3,https://en.wikipedia.org/wiki/DBAG_Class_403,"ICE 3\n, or \nIntercity-Express 3\n, is a fami...",DB/NS Class 403/406/407\nICE 3 on the \nCologn...
1104,Train,https://en.wikipedia.org/wiki/Train,\n\n\nA \ntrain\n is a form of \nrail transpor...,Part of a series on\nRail transport\n\n\nOpera...
1136,Toronto streetcar system,https://en.wikipedia.org/wiki/1495_mm_gauge_ra...,\n\n\n\nThe \nToronto streetcar system\n is a ...,Toronto streetcar system\nFlexity Outlook\n st...


In [39]:
table[table.duplicated(subset=['URL'])]

,Title,URL,Text,Table Table


In [40]:
table[table.Title=="Revenue"]

,Title,URL,Text,Table Table
4151,Revenue,https://en.wikipedia.org/wiki/Net_revenue,"In \naccounting\n, \nrevenue\n is the income t...",Part of \na series\n on\nAccounting\n\n\nHisto...
11186,Revenue,https://en.wikipedia.org/wiki/Revenues,"In \naccounting\n, \nrevenue\n is the income t...",Part of \na series\n on\nAccounting\n\n\nHisto...


In [41]:
table[table.Title=="List of "]

,Title,URL,Text,Table Table
527,List of,https://en.wikipedia.org/wiki/List_of_American...,This article lists characters from the \nanima...,\n\n\n\n\nCast members\n\n\n\n\n\n\n\n\n\n\n\n...
4599,List of,https://en.wikipedia.org/wiki/List_of_Mayday_e...,"\n\n\nMayday\n, known as \nAir Crash Investiga...",Seasons\n[1]\nEpisodes\nOriginally aired\nFirs...
5923,List of,https://en.wikipedia.org/wiki/Phil_and_Lil_DeV...,\n\nRugrats\n includes a large array of charac...,This article has multiple issues.\n Please hel...
6577,List of,https://en.wikipedia.org/wiki/Grolim,This is a \nlist of \nThe Belgariad\n and \nTh...,v\nt\ne\nBibliography\n by \nDavid Eddings\nBe...
6598,List of,https://en.wikipedia.org/wiki/Aldur,This is a \nlist of \nThe Belgariad\n and \nTh...,v\nt\ne\nBibliography\n by \nDavid Eddings\nBe...
6607,List of,https://en.wikipedia.org/wiki/Chaldan,This is a \nlist of \nThe Belgariad\n and \nTh...,v\nt\ne\nBibliography\n by \nDavid Eddings\nBe...
9035,List of,https://en.wikipedia.org/wiki/List_of_Samantha...,The following is a list of characters in the \...,This article \ndoes not \ncite\n any \nsources...
11404,List of,https://en.wikipedia.org/wiki/Gale_Hawthorne,The following is a list of characters in \nThe...,This article \nneeds additional citations for ...


In [54]:
table.drop_duplicates(subset=['Title'],inplace=True)

In [58]:
table

,Title,URL,Text,Table Table
0,Virus,https://en.wikipedia.org/wiki/Virus,\n\n\n\nSee text\n\n\nA \nvirus\n is a submicr...,\n\n\n\n\nVirus\n\n\n\n\n\n\n\n\n\n\n\nSARS-Co...
1,Morphology (biology),https://en.wikipedia.org/wiki/Morphology_(biol...,Morphology\n is a branch of \nbiology\n dealin...,v\nt\ne\nAnatomy\n and \nmorphology\nFields\n\...
2,Pleomorphism (microbiology),https://en.wikipedia.org/wiki/Pleomorphism_(mi...,"In microbiology, \npleomorphism\n (from \nAnci...",NaN
3,Respiratory tract infection,https://en.wikipedia.org/wiki/Respiratory_trac...,A \nrespiratory tract infection\n (\nRTI\n) is...,Respiratory tract infection\nConducting passag...
4,Peplomer,https://en.wikipedia.org/wiki/Peplomer,A \npeplomer\n is a \nglycoprotein\n spike on ...,This article \nneeds additional citations for ...
5,Electron microscope,https://en.wikipedia.org/wiki/Electron_microscopy,An \nelectron microscope\n is a \nmicroscope\n...,\n\n\n\nSemiconductor and data storage\n\n\nCi...
6,Human coronavirus HKU1,https://en.wikipedia.org/wiki/Human_coronaviru...,Human coronavirus HKU1\n (\nHCoV-HKU1\n) is a ...,\n\n\n\n\nHuman coronavirus HKU1\n\n\n\n\n\n\n...
7,Ancient Greek,https://en.wikipedia.org/wiki/Ancient_Greek,\n\n\n\nAncient Greek\n includes the forms of ...,This article \nneeds additional citations for ...
9,Human coronavirus NL63,https://en.wikipedia.org/wiki/Human_coronaviru...,Human coronavirus NL63\n (\nHCoV-NL63\n) is a ...,\n\n\n\n\nHuman coronavirus NL63\n\n\n\n\n\n\n...
10,Severe acute respiratory syndrome coronavirus,https://en.wikipedia.org/wiki/Severe_acute_res...,\n\n\n\nSevere acute respiratory syndrome coro...,\n\n\n\n\nSevere acute respiratory syndrome co...


In [55]:
table[table.duplicated(subset=['Text'])]

,Title,URL,Text,Table Table
10973,Talk:Political history of the world,https://en.wikipedia.org/wiki/Talk:Political_h...,Navigation menu,\n\nThis is the \ntalk page\n for discussing i...


In [45]:
table.URL[10973]

'https://en.wikipedia.org/wiki/Talk:Political_history_of_the_world'

In [56]:
table[table.Text=='Navigation menu']

,Title,URL,Text,Table Table
11,Encapsidation,https://en.wikipedia.org/wiki/Encapsidation,Navigation menu,Wikipedia does not have an article with this e...
10973,Talk:Political history of the world,https://en.wikipedia.org/wiki/Talk:Political_h...,Navigation menu,\n\nThis is the \ntalk page\n for discussing i...


In [57]:
table = table[table.Text != "Navigation menu"]

In [61]:
table = table[table.Text.map(len)>500]

In [63]:
# confirming
table[table.Text.map(len)<500]

,Title,URL,Text,Table Table
69,Hexokinase deficiency,https://en.wikipedia.org/wiki/Hexokinase_defic...,Hexokinase deficiency\n is an \nanemia\n-causi...,Hexokinase deficiency\nSpecialty\nHematology\n...
82,Interolog,https://en.wikipedia.org/wiki/Interolog,An \ninterolog\n is a conserved interaction be...,\nThis bioinformatics-related article is a \n...
144,"St John the Baptist's Church, Ault Hucknall",https://en.wikipedia.org/wiki/St_John_the_Bapt...,"St John the Baptist’s Church, Ault Hucknall\n ...","St John the Baptist’s Church, Ault Hucknall\nS..."
188,Simon Campbell,https://en.wikipedia.org/wiki/Simon_Campbell,"Sir Simon Fraser Campbell\n, \nCBE\n,\n \nFRS\...","Sir\nSimon Campbell\nCBE\n,\n \nFRS\n,\n \nFMe..."
230,Network address,https://en.wikipedia.org/wiki/Network_address,A \nnetwork address\n is an identifier for a \...,This \ncomputer networking\n article is a \nst...
247,Royal Antigua and Barbuda Defence Force,https://en.wikipedia.org/wiki/Royal_Antigua_an...,\n\n\nThe \nRoyal Antigua and Barbuda Defence ...,This article \nneeds additional citations for ...
275,Princess Mabereng Seeiso of Lesotho,https://en.wikipedia.org/wiki/Princess_Maberen...,Princess Mabereng Seeiso of Lesotho\n (born 'M...,Princess Marebeng\nPrincess of Lesotho\nSpouse...
295,2010 Toronto International Film Festival,https://en.wikipedia.org/wiki/2010_Toronto_Int...,The 35th annual \nToronto International Film F...,2010 Toronto International Film Festival\nFest...
316,Paralympic powerlifting,https://en.wikipedia.org/wiki/Paralympic_power...,Paralympic powerlifting\n is an adaptation of ...,\n\n\n\nCategory\n\n\nRecord\n\n\nAthlete\n\n\...
349,Intégrale : Institut d'enseignement supérieur ...,https://en.wikipedia.org/wiki/Int%C3%A9grale_:...,Intégrale : Institut d'enseignement supérieur ...,Intégrale : Institut d'enseignement supérieur ...


In [80]:
table.Text[4771]

'Contents\nL\n[\nedit\n]\n\n\nReferences\n[\nedit\n]\nNavigation menu'

In [81]:
table.URL[10922]

'https://en.wikipedia.org/wiki/Smash_and_Dash_(disambiguation)'

In [84]:
import re

In [134]:
text = '''
https://en.wikipedia.org/wiki/Smash_and_Dash_(disambiguation)
https://en.wikipedia.org/wiki/Category:Cuisine
https://en.wikipedia.org/wiki/Talk:Cuisine
'''

In [95]:
print(re.split('[https://en.wikipedia.org/wiki/+a-zA-Z0-9:]'," ",text))

TypeError: 'str' object cannot be interpreted as an integer

In [101]:
print(re.sub('[https://en.wikipedia.org/wiki/][a-zA-Z0-9]+[:] '," ",text))


https://en.wikipedia.org/wiki/Smash_and_Dash_(disambiguation)
https://en.wikipedia.org/wiki/Category:Cuisine
https://en.wikipedia.org/wiki/Talk:Cuisine



In [117]:
st = text.split()[1]

In [112]:
st2 = st.split('https://en.wikipedia.org/wiki/')[1]

In [113]:
st2.split(":")

['Smash_and_Dash_(disambiguation)']

In [118]:
re.findall(':',st)

[':', ':']

In [145]:
re.findall('https://en.wikipedia.org/wiki/[\w]+',text)

['https://en.wikipedia.org/wiki/Smash_and_Dash_',
 'https://en.wikipedia.org/wiki/Category',
 'https://en.wikipedia.org/wiki/Talk']

In [149]:
re.findall('[\w]+:',text)

['https:', 'https:', 'Category:', 'https:', 'Talk:']

In [150]:
reg = re.compile("https://en.wikipedia.org/wiki/.+:")

In [152]:
re.match(reg, text)

In [153]:
re.findall(reg,text)

['https://en.wikipedia.org/wiki/Category:',
 'https://en.wikipedia.org/wiki/Talk:']

In [161]:
re.findall('https://en.wikipedia.org/wiki/[a-zA-Z0-9_()]+',text)

['https://en.wikipedia.org/wiki/Smash_and_Dash_(disambiguation)',
 'https://en.wikipedia.org/wiki/Category',
 'https://en.wikipedia.org/wiki/Talk']

In [64]:
table['Table Table'] = table['Table Table'].map(str)

In [65]:
table.Text = table.Text.str.strip()
table['Table Table'] = table['Table Table'].str.strip()

In [66]:
def initial_clean(text):
    """
    Function to clean text of websites, email addresess and any punctuation
    We also lower case the text
    """
    pattern = r"((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)"
    text = re.sub('[[a-zA-Z0-9]+]|[\n[\nedit\n]+]',' ',text)
    text = re.sub(pattern," ",text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = text.lower()
    text = nltk.word_tokenize(text)
    return text

stop_words = STOPWORDS

def remove_stop_words(text):
    '''
    Function to remove stopwords from text
    '''
    return [word for word in text if word not in stop_words]

def lemmatize(text):
    '''
    Function to perform lemmatization on text
    '''
    lemmed = [WordNetLemmatizer().lemmatize(word) for word in text if len(word)>2]
    lemmed = [WordNetLemmatizer().lemmatize(word,pos='v') for word in lemmed]
    return lemmed

def apply_all(text):
    return lemmatize(remove_stop_words(initial_clean(text)))

In [67]:
# clean text and title and create new column "tokenized"
t1 = time.time()
table['tokenized'] = table['Text'].apply(apply_all) + table['Table Table'].apply(apply_all)
t2 = time.time()
print("Time to clean and tokenize", len(table), "articles:", (t2-t1)/60, "min")

Time to clean and tokenize 10909 articles: 6.4215967655181885 min


In [68]:
table.head()

,Title,URL,Text,Table Table,tokenized
0,Virus,https://en.wikipedia.org/wiki/Virus,See text\n\n\nA \nvirus\n is a submicroscopic ...,"Virus\n\n\n\n\n\n\n\n\n\n\n\nSARS-CoV-2, a mem...","[text, virus, submicroscopic, infectious, agen..."
1,Morphology (biology),https://en.wikipedia.org/wiki/Morphology_(biol...,Morphology\n is a branch of \nbiology\n dealin...,v\nt\ne\nAnatomy\n and \nmorphology\nFields\n\...,"[morphology, branch, biology, deal, study, for..."
2,Pleomorphism (microbiology),https://en.wikipedia.org/wiki/Pleomorphism_(mi...,"In microbiology, \npleomorphism\n (from \nAnci...",nan,"[microbiology, pleomorphism, ancient, greek, m..."
3,Respiratory tract infection,https://en.wikipedia.org/wiki/Respiratory_trac...,A \nrespiratory tract infection\n (\nRTI\n) is...,Respiratory tract infection\nConducting passag...,"[respiratory, tract, infection, rti, number, i..."
5,Electron microscope,https://en.wikipedia.org/wiki/Electron_microscopy,An \nelectron microscope\n is a \nmicroscope\n...,Semiconductor and data storage\n\n\nCircuit ed...,"[electron, microscope, microscope, us, beam, a..."


In [69]:
# list of all words
all_words = [word for item in list(table['tokenized']) for word in item]
# use nltk fdist to get a frequency distribution of all words
fdist = FreqDist(all_words)
len(fdist) # number of unique words

368797

In [70]:
import pickle

In [71]:
pickle.dump(fdist,open("fdist_file2.pkl","wb"))

In [188]:
fdist

FreqDist({'edit': 77330, 'state': 58562, 'new': 47549, 'year': 39473, 'time': 36003, 'include': 34387, 'city': 31999, 'war': 30971, 'unite': 30670, 'group': 24353, ...})

In [72]:
fdist

FreqDist({'state': 58380, 'new': 47466, 'year': 39333, 'time': 35958, 'include': 34350, 'city': 31916, 'war': 30863, 'unite': 30571, 'group': 24311, 'national': 23535, ...})

In [83]:
k = 100000
top_k_words = fdist.most_common(k)
top_k_words[-10:]

[('gigue', 5),
 ('surtitle', 5),
 ('humperdinck', 5),
 ('divertissement', 5),
 ('orphean', 5),
 ('protour', 5),
 ('lemond', 5),
 ('equipe', 5),
 ('kimmage', 5),
 ('uec', 5)]

In [84]:
# function to keep only top k words
top_k_words,_ = zip(*fdist.most_common(k))
top_k_words = set(top_k_words)

def keep_top_k_words(text):
    return [word for word in text if word in top_k_words]


In [85]:
table['tokenized'] = table['tokenized'].apply(keep_top_k_words)

In [86]:
# document length
table['doc_len'] = table['tokenized'].apply(lambda x: len(x))
doc_lengths = list(table['doc_len'])
table.drop(labels='doc_len', axis=1, inplace=True)

print("length of list:",len(doc_lengths),
      "\naverage document length", np.average(doc_lengths),
      "\nminimum document length", min(doc_lengths),
      "\nmaximum document length", max(doc_lengths))

length of list: 10909 
average document length 1293.3463195526629 
minimum document length 21 
maximum document length 24238


In [88]:
corpus = list(table['tokenized'])

In [89]:
pickle.dump(corpus,open("wiki_corpus_file.pkl","wb"))

In [94]:
wiki_title_file = list(table["Title"])
wiki_text_file = list(table["Text"])

In [95]:
wiki_title_file[10]

'Severe acute respiratory syndrome coronavirus 2'

In [96]:
pickle.dump(wiki_title_file,open("wiki_title_file.pkl","wb"))
pickle.dump(wiki_text_file,open("wiki_text_file.pkl","wb"))

In [18]:
table = pd.read_csv("wikipedia_articles2.csv")

In [19]:
table.head()

,Title,URL,Text,Table Text
0,Virus,https://en.wikipedia.org/wiki/Virus,\n\n\n\nSee text\n\n\nA \nvirus\n is a submicr...,\n\n\n\n\nVirus\n\n\n\n\n\n\n\n\n\n\n\nSARS-Co...
1,Severe acute respiratory syndrome coronavirus 2,https://en.wikipedia.org/wiki/SARS-CoV-2,\n\n\n\nSevere acute respiratory syndrome coro...,\n\n\n\n\nSevere acute respiratory syndrome co...
2,Middle East respiratory syndrome-related coron...,https://en.wikipedia.org/wiki/Middle_East_resp...,Middle East respiratory syndrome-related coron...,\n\n\n\n\nMiddle East respiratory syndrome-rel...
3,Human coronavirus HKU1,https://en.wikipedia.org/wiki/Human_coronaviru...,Human coronavirus HKU1\n (\nHCoV-HKU1\n) is a ...,\n\n\n\n\nHuman coronavirus HKU1\n\n\n\n\n\n\n...
4,Transmissible gastroenteritis virus,https://en.wikipedia.org/wiki/Transmissible_ga...,Transmissible gastroenteritis virus\n or \nTra...,This article includes a \nlist of references\n...


In [26]:
# table[table.duplicated(subset=['Title','Text'])]
table.drop_duplicates(subset=['Title','Text'],inplace=True)

In [35]:
# table[table.Text.map(len)<400]
# print(table.Text[3642])
print(table.URL[4759])
print(table.Text[8851])

https://en.wikipedia.org/wiki/List_of_Masters_of_Jesus_College,_Cambridge
References
[
edit
]


Navigation menu
Personal tools
Namespaces

            		
Variants

            	
Views

            		
More

            	

            		
Search

            	

    			Navigation
    		

    			Interaction
    		

    			Tools
    		

    			Print/export
    		

    			Languages
    		


In [11]:
print(table.Text[0].strip())

See text


A 
virus
 is a submicroscopic 
infectious agent
 that 
replicates
 only inside the living 
cells
 of an 
organism
. Viruses can infect all types of 
life forms
, from animals and plants to 
microorganisms
, including 
bacteria
 and 
archaea
.
[1]

Since 
Dmitri Ivanovsky
's 1892 article describing a non-bacterial 
pathogen
 infecting tobacco plants, and the discovery of the 
tobacco mosaic virus
 by 
Martinus Beijerinck
 in 1898,
[2]
 about 5,000  
virus species
 have been described in detail,
[3]
 of the millions of types of viruses in the environment.
[4]
 Viruses are found in almost every 
ecosystem
 on Earth and are the most numerous type of biological entity.
[5]
[6]
 The study of viruses is known as 
virology
, a subspeciality of 
microbiology
.

When not inside an infected cell or in the process of infecting a cell, viruses exist in the form of independent particles, or 
virions
, consisting of: (i) the 
genetic material
, i.e. long 
molecules
 of 
DNA
 or 
RNA
 that 

In [13]:
textt = table.Text[4000]
# re.findall('[[a-zA-Z0-9]+]',textt)
print(re.sub('[[a-zA-Z0-9]+]|[\n[\nedit\n]+]',' ',textt))
# re.findall('[\n[\nedit\n]+]',textt)
# re.sub('[\n[\nedit\n]+]',' ',textt)


The 
Discourses and Mathematical Demonstrations Relating to Two New Sciences
 (
Italian
: 
Discorsi e dimostrazioni matematiche intorno a due nuove scienze
 
pronounced 
[diˈskorsi e dimostratˈtsjoːni mateˈmaːtike inˈtorno a ˈduːe ˈnwɔːve ˈʃɛ 
) published in 1638 was 
Galileo's
 final book and a scientific testament covering much of his work in 
physics
 over the preceding thirty years.

After his 
Dialogue Concerning the Two Chief World Systems
, the 
Roman Inquisition
 had banned the publication of any of Galileo's works, including any he might write in the future.
 
 After the failure of his initial attempts to publish 
Two New Sciences
 in 
France
, 
Germany
, and 
Poland
, it was published by 
Lodewijk Elzevir
 who was working in 
Leiden
, 
South Holland
, where the writ of the Inquisition was of less consequence (see 
House of Elzevir
).
 
 Fra Fulgenzio Micanzio, the official theologian of the Republic of Venice, had initially offered to help Galileo publish in Venice the new wo

In [14]:
table['Table Text'] = table['Table Text'].map(str)
table['Table Text'] = table['Table Text'].str.strip()

table.Text = table.Text.str.strip()
table.Text = table.Text.map(str)

In [7]:
def initial_clean(text):
    """
    Function to clean text which can contain Links, email addresess, and any punctuation
    We also lower case the text
    """
    pattern = r"((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)"
    text = re.sub(pattern," ",text)
    text = re.sub('[[a-zA-Z0-9]+]|[\n[\nedit\n]+]',' ',text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = text.lower()
    text = nltk.word_tokenize(text)
    return text

stop_words = STOPWORDS

def remove_stop_words(text):
    '''
    Function to remove stopwords from text
    '''
    return [word for word in text if word not in stop_words]

def lemmatize(text):
    '''
    Function to perform lemmatization on text
    '''
    lemmed = [WordNetLemmatizer().lemmatize(word) for word in text if len(word)>2]
    lemmed = [WordNetLemmatizer().lemmatize(word,pos='v') for word in lemmed]
    return lemmed

def apply_all(text):
    return lemmatize(remove_stop_words(initial_clean(text)))

In [ ]:
# clean text and title and create new column "tokenized"
t1 = time.time()
table['tokenized'] = table['Text'].apply(apply_all) + table['Table Table'].apply(apply_all)
t2 = time.time()
print("Time to clean and tokenize", len(table), "articles:", (t2-t1)/60, "min")